# PUBG: PyTorch

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore')

from copy import deepcopy

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import gc, sys
gc.enable()

import os
print(os.listdir("./input"))

['test.csv', 'train.csv']


## Framework

### Common functions

In [2]:
# Thanks to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#        else:
#            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Data preparation

In [3]:
def take_part_of_data(df, part):
    
    match_ids = df['matchId'].unique()
    match_ids_part = np.random.choice(match_ids, int(part * len(match_ids)))
    
    df = df[df['matchId'].isin(match_ids_part)]
    
    del match_ids
    del match_ids_part

### Feature engineering

In [4]:
def add_new_features_1(df):
    
    # calculate total distance
    df['totalDistance'] = df['rideDistance'] + df['walkDistance'] + df['swimDistance']
    
    # sum heals and boosts
    df['healsAndBoosts'] = df['heals'] + df['boosts']
    
    # headshot rate
    df['headshotKillsOverKills'] = df['headshotKills'] / df['kills']
    df['headshotKillsOverKills'].fillna(0, inplace=True)
    
    # kill streake rate
    df['killStreaksOverKills'] = df['killStreaks'] / df['kills']
    df['killStreaksOverKills'].fillna(0, inplace=True)
    
    # kills and assists
    df['killsAndAssists'] = df['kills'] + df['assists']
    
    # teamwork
    df['assistsAndRevives'] = df['assists'] + df['revives']

In [5]:
def add_new_features_2(df):
    
    # number of players joined
    df['playersJoined'] = df.groupby('matchId')['matchId'].transform('count')
    
    # normalize features by number of players joined
    df['killsAndAssistsOverPlayersJoined'] = df['killsAndAssists'] * ((100 - df['playersJoined']) / 100 + 1)
    df['matchDurationOverPlayersJoined'] = df['matchDuration'] * ((100 - df['playersJoined']) / 100 + 1)
    df['damageDealtOverPlayersJoined'] = df['damageDealt'] * ((100 - df['playersJoined']) / 100 + 1)

In [6]:
def add_new_features_3(df):
    
    # total distance over kills and assists
    df['totalDistanceOverKillsAndAssists'] = df['totalDistance'] / df['killsAndAssists']
    df['totalDistanceOverKillsAndAssists'].fillna(0, inplace=True)
    df['totalDistanceOverKillsAndAssists'].replace(np.inf, 0, inplace=True)
    
    # total distance over heals and boosts
    df['totalDistanceOverHealsAndBoosts'] = df['totalDistance'] / df['healsAndBoosts']
    df['totalDistanceOverHealsAndBoosts'].fillna(0, inplace=True)
    df['totalDistanceOverHealsAndBoosts'].replace(np.inf, 0, inplace=True)

In [7]:
def add_new_features_4(df):
    
    df['headshotRate'] = df['kills'] / df['headshotKills']
    df['killStreakRate'] = df['killStreaks'] / df['kills']
    df['healsAndBoosts'] = df['heals'] + df['boosts']
    df['totalDistance'] = df['rideDistance'] + df['walkDistance'] + df['swimDistance']
    df['killPlaceOverMaxPlace'] = df['killPlace'] / df['maxPlace']
    df['headshotKillsOverKills'] = df['headshotKills'] / df['kills']
    df['distanceOverWeapons'] = df['totalDistance'] / df['weaponsAcquired']
    df['walkDistanceOverHeals'] = df['walkDistance'] / df['heals']
    df['walkDistanceOverKills'] = df['walkDistance'] / df['kills']
    df['killsPerWalkDistance'] = df['kills'] / df['walkDistance']
    df["skill"] = df['headshotKills'] + df['roadKills']
    
    df[df == np.Inf] = np.NaN
    df[df == np.NINF] = np.NaN
    
    df.fillna(0, inplace=True)

In [8]:
def feature_engineering(df, is_train=True):
    
    # fix rank points
    df['rankPoints'] = np.where(df['rankPoints'] <= 0, 0, df['rankPoints'])
    
    features = list(df.columns)
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    if 'winPlacePerc' in features:
        features.remove('winPlacePerc')
    
    y = None
    
    # average y for training dataset
    if is_train:
        y = df.groupby(['matchId','groupId'])['winPlacePerc'].agg('mean')
    elif 'winPlacePerc' in df.columns:
        y = df['winPlacePerc']
    
    # mean by match and group
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    if is_train:
        df_out = agg.reset_index()[['matchId','groupId']]
    else:
        df_out = df[['matchId','groupId']]
    
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    # max by match and group
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    # max by match and group
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    # number of players in group
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
    
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    # mean by match
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    # number of groups in match
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    # drop match id and group id
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)
    
    del agg, agg_rank
    
    return df_out, y

### Machine learning

In [9]:
class Estimator(object):
    
    def fit(self, x_train, y_train, x_valid, y_valid):
        raise NotImplementedException
    
    def predict(self, x):
        raise NotImplementedException

In [10]:
class ScikitLearnEstimator(Estimator):
    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, x_train, y_train, x_valid, y_valid):
        self.estimator.fit(x_train, y_train)
    
    def predict(self, x):
        return self.estimator.predict(x)

In [11]:
def fit_predict_step(estimator, x_train, y_train, train_idx, valid_idx, x_test, oof):
    
    # prepare train and validation data
    x_train_train = x_train[train_idx]
    y_train_train = y_train[train_idx]
    x_train_valid = x_train[valid_idx]
    y_train_valid = y_train[valid_idx]
    
    # fit estimator
    estimator.fit(x_train_train, y_train_train, x_train_valid, y_train_valid)
    
    # collect OOF
    oof_part = estimator.predict(x_train_valid)
    
    print('MAE:', mean_absolute_error(y_train_valid, oof_part))
    
    oof[valid_idx] = oof_part
    
    # make predictions for test data
    y_part = estimator.predict(x_test)
    
    return y_part

In [12]:
def fit_predict(estimator, x_train, y_train, x_test):
    
    oof = np.zeros(x_train.shape[0])
    
    y = np.zeros(x_test.shape[0])
    
    kf = KFold(n_splits=5, random_state=42)
    
    for train_idx, valid_idx in kf.split(x_train):
        
        y_part = fit_predict_step(estimator, x_train, y_train, train_idx, valid_idx, x_test, oof)
        
        # average predictions for test data
        y += y_part / kf.n_splits
    
    print('Final MAE:', mean_absolute_error(y_train, oof))
    
    return oof, y

In [13]:
def fit_step(estimator, x_train, y_train, train_idx, valid_idx, oof):
    
    # prepare train and validation data
    x_train_train = x_train[train_idx]
    y_train_train = y_train[train_idx]
    x_train_valid = x_train[valid_idx]
    y_train_valid = y_train[valid_idx]
    
    # fit estimator
    estimator.fit(x_train_train, y_train_train, x_train_valid, y_train_valid)
    
    # collect OOF
    oof_part = estimator.predict(x_train_valid)
    
    mae = mean_absolute_error(y_train_valid, oof_part)
    print('MAE:', mae)
    
    oof[valid_idx] = oof_part
    
    return estimator, mae

In [14]:
def fit(estimator, x_train, y_train):
    
    oof = np.zeros(x_train.shape[0])
    
    kf = KFold(n_splits=5, random_state=42)
    
    trained_estimators = []
    
    for train_idx, valid_idx in kf.split(x_train):
        
        e, mae = fit_step(estimator, x_train, y_train, train_idx, valid_idx, oof)
        
        trained_estimators.append(deepcopy(e))
    
    print('Final MAE:', mean_absolute_error(y_train, oof))
    
    return oof, trained_estimators

In [15]:
def predict(trained_estimators, x_test):
    
    y = np.zeros(x_test.shape[0])
    
    for estimator in trained_estimators:
        
        y_part = estimator.predict(x_test)
        
        # average predictions for test data
        y += y_part / len(trained_estimators)
    
    return y

In [16]:
def pipeline_fit(estimator, df_train, scaler=None):
    
    # add new features
    add_new_features_4(df_train)
    
    # feature engineering
    x_train, y_train = feature_engineering(df_train, is_train=True)
    x_train = reduce_mem_usage(x_train)
    gc.collect()
    
    # scale
    if not (scaler is None):
        scaler.fit(x_train)
        scaled_x_train = scaler.transform(x_train)
    else:
        scaled_x_train = x_train.values
    
    # fit
    oof, trained_estimators = fit(estimator, scaled_x_train, y_train.values)
    
    del x_train
    del scaled_x_train
    del y_train
    gc.collect()
    
    return oof, trained_estimators

In [17]:
def pipeline_predict(trained_estimators, df_test, scaler=None):
    
    # add new features
    add_new_features_4(df_test)
    
    # feature engineering
    x_test, _ = feature_engineering(df_test, is_train=False)
    x_test = reduce_mem_usage(x_test)
    gc.collect()
    
    # scale
    if not (scaler is None):
        scaled_x_test = scaler.transform(x_test)
    else:
        scaled_x_test = x_test.values
    
    # predict
    y = predict(trained_estimators, scaled_x_test)
    
    del x_test
    del scaled_x_test
    gc.collect()
    
    return y

## Load train data

In [19]:
df_train = pd.read_csv('./input/train.csv', index_col='Id')
df_train.shape

(7134, 28)

In [20]:
df_train = reduce_mem_usage(df_train)

Memory usage of dataframe is 1.58 MB
Memory usage after optimization is: 0.46 MB
Decreased by 71.1%


In [21]:
df_train.head().T

Id,2860d3f9ee4d4b,0b88c47e54cdcc,8f086d61515bc4,7c037ca62a38a4,82f08cde238c60
groupId,46f1b1c3bf0dae,5e92598b08bcb1,ed4cfbc560f490,d2ff4830704e19,7c5002766d8c7a
matchId,25ade93a241f39,95125f5e05c653,8a84da96d077c3,a139eac809980a,e9d203a36b1d0a
assists,0,0,0,0,0
boosts,0,2,0,0,0
damageDealt,0,359,223.5,128.125,200
DBNOs,0,4,1,0,2
headshotKills,0,0,0,0,0
heals,0,0,0,0,0
killPlace,80,5,15,50,36
killPoints,0,1387,1673,1479,0


In [22]:
gc.collect()

20

## Remove bad row

In [23]:
df_train.drop(df_train[df_train['winPlacePerc'].isnull()].index, inplace=True)

## PyTorch

In [24]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.nn.utils.weight_norm import weight_norm

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [26]:
class PyTorch(Estimator):
    
    def fit(self, x_train, y_train, x_valid, y_valid):
        
        train_tensor = TensorDataset(
            torch.from_numpy(x_train.astype('float32')),
            torch.from_numpy(y_train.astype('float32')))
        train_loader = DataLoader(train_tensor, batch_size=256, shuffle=True)
        
        self.model = nn.Sequential(
            weight_norm(nn.Linear(x_train.shape[1], 128)),
            nn.ReLU(),
            weight_norm(nn.Linear(128, 128)),
            nn.ReLU(),
            weight_norm(nn.Linear(128, 128)),
            nn.ReLU(),
            weight_norm(nn.Linear(128, 128)),
            nn.ReLU(),
            weight_norm(nn.Linear(128, 1))).to(device)
        
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight_v)
                nn.init.kaiming_normal_(m.weight_g)
                nn.init.constant_(m.bias, 0)
        
        criterion = nn.L1Loss()
        optimizer = torch.optim.Adam(self.model.parameters(), betas=(0.9, 0.999), lr=1e-3)

        self.model.train()
        n_epochs = 50
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for train_part, y_part in train_loader:
                optimizer.zero_grad()
                y_pred_part = self.model(train_part.to(device))
                loss = criterion(y_pred_part.reshape(-1), y_part.to(device))
                loss.backward()
                optimizer.step()
                epoch_loss += y_pred_part.shape[0] * loss.item()
            print('Epoch %3d / %3d. Loss = %.5f' % (epoch + 1, n_epochs, epoch_loss / x_train.shape[0]))
    
    def predict(self, x):
        self.model.eval()
        x_tensor = torch.from_numpy(x.astype('float32'))
        x_loader = DataLoader(x_tensor, batch_size=256, shuffle=False)
        y_pred = np.empty(0)
        with torch.no_grad():
            for x_part in x_loader:
                y_pred_part = self.model(x_part.to(device)).data.cpu().numpy().reshape(-1)
                y_pred = np.append(y_pred, y_pred_part)
        return y_pred

In [27]:
%%time

scaler = StandardScaler()
oof, trained_estimators = pipeline_fit(PyTorch(), df_train, scaler)

Memory usage of dataframe is 4.37 MB
Memory usage after optimization is: 1.40 MB
Decreased by 68.0%
Epoch   1 /  50. Loss = 0.41332
Epoch   2 /  50. Loss = 0.19249
Epoch   3 /  50. Loss = 0.12973
Epoch   4 /  50. Loss = 0.10432
Epoch   5 /  50. Loss = 0.09207
Epoch   6 /  50. Loss = 0.07813
Epoch   7 /  50. Loss = 0.06677
Epoch   8 /  50. Loss = 0.06249
Epoch   9 /  50. Loss = 0.05666
Epoch  10 /  50. Loss = 0.05237
Epoch  11 /  50. Loss = 0.05293
Epoch  12 /  50. Loss = 0.05044
Epoch  13 /  50. Loss = 0.04986
Epoch  14 /  50. Loss = 0.04826
Epoch  15 /  50. Loss = 0.04412
Epoch  16 /  50. Loss = 0.04132
Epoch  17 /  50. Loss = 0.04443
Epoch  18 /  50. Loss = 0.04310
Epoch  19 /  50. Loss = 0.04151
Epoch  20 /  50. Loss = 0.04226
Epoch  21 /  50. Loss = 0.03734
Epoch  22 /  50. Loss = 0.03805
Epoch  23 /  50. Loss = 0.03566
Epoch  24 /  50. Loss = 0.04046
Epoch  25 /  50. Loss = 0.04125
Epoch  26 /  50. Loss = 0.04053
Epoch  27 /  50. Loss = 0.03478
Epoch  28 /  50. Loss = 0.03420
Epoc

Wall time: 43.6 s


In [28]:
del df_train

gc.collect()

20

## Load test data

In [29]:
df_test = pd.read_csv('./input/test.csv', index_col = 'Id')
df_test.shape

(1794, 27)

In [30]:
df_test = reduce_mem_usage(df_test)

Memory usage of dataframe is 0.38 MB
Memory usage after optimization is: 0.11 MB
Decreased by 71.0%


In [31]:
df_test_id = pd.DataFrame(index=df_test.index)

In [32]:
gc.collect()

80

## PyTorch

In [33]:
y = pipeline_predict(trained_estimators, df_test, scaler)

Memory usage of dataframe is 2.42 MB
Memory usage after optimization is: 0.77 MB
Decreased by 68.0%


In [34]:
del df_test

gc.collect()

20

## Save OOF

In [35]:
df_oof = pd.DataFrame()
df_oof['pytorch_oof'] = oof
df_oof.to_csv('pytorch_oof.csv', index_label='id')

## Submission

In [36]:
df_submission = pd.DataFrame(index=df_test_id.index)
df_submission['winPlacePerc'] = y
df_submission.to_csv('pytorch_raw.csv', index_label='Id')

## Adjust predictions

In [37]:
df_test = pd.read_csv('./input/test.csv')
df_test.shape

(1794, 28)

In [38]:
df_submission = df_submission.merge(df_test[['Id', 'matchId', 'groupId', 'maxPlace', 'numGroups']], on='Id', how='left')
df_submission.head()

,Id,winPlacePerc,matchId,groupId,maxPlace,numGroups
0,c9ce5e5749758a,0.322456,6533442a513d6f,d1d34623858d02,29,27
1,262418e6a1fd89,0.166025,976f4ae3c91669,f0290ae9582198,49,44
2,1fdaad049557d9,0.322456,6533442a513d6f,d1d34623858d02,29,27
3,cb1761d22b957e,0.286276,efd749a746587e,77234d376925f9,50,47
4,1b86424e60c2e8,0.558887,098d3f2da942e7,19a7b7ce87c841,50,47


In [39]:
df_submission_group = df_submission.groupby(['matchId', 'groupId']).first().reset_index()

df_submission_group['rank'] = df_submission_group.groupby(['matchId'])['winPlacePerc'].rank()

df_submission_group = df_submission_group.merge(df_submission_group.groupby('matchId')['rank'].max().to_frame('max_rank').reset_index(), on='matchId', how='left')

df_submission_group['adjusted_perc'] = (df_submission_group['rank'] - 1) / (df_submission_group['numGroups'] - 1)

df_submission = df_submission.merge(df_submission_group[['adjusted_perc', 'matchId', 'groupId']], on=['matchId', 'groupId'], how='left')

df_submission['winPlacePerc'] = df_submission['adjusted_perc']

df_submission.head()

,Id,winPlacePerc,matchId,groupId,maxPlace,numGroups,adjusted_perc
0,c9ce5e5749758a,0.307692,6533442a513d6f,d1d34623858d02,29,27,0.307692
1,262418e6a1fd89,0.186047,976f4ae3c91669,f0290ae9582198,49,44,0.186047
2,1fdaad049557d9,0.307692,6533442a513d6f,d1d34623858d02,29,27,0.307692
3,cb1761d22b957e,0.304348,efd749a746587e,77234d376925f9,50,47,0.304348
4,1b86424e60c2e8,0.565217,098d3f2da942e7,19a7b7ce87c841,50,47,0.565217


In [40]:
df_submission.loc[df_submission.maxPlace == 0, 'winPlacePerc'] = 0
df_submission.loc[df_submission.maxPlace == 1, 'winPlacePerc'] = 1

In [41]:
# Thanks to https://www.kaggle.com/anycode/simple-nn-baseline-4
t = df_submission.loc[df_submission.maxPlace > 1]
gap = 1.0 / (t.maxPlace.values - 1)
fixed_perc = np.around(t.winPlacePerc.values / gap) * gap
df_submission.loc[df_submission.maxPlace > 1, 'winPlacePerc'] = fixed_perc

In [42]:
df_submission.loc[(df_submission.maxPlace > 1) & (df_submission.numGroups == 1), 'winPlacePerc'] = 0

assert df_submission['winPlacePerc'].isnull().sum() == 0

In [43]:
df_submission[['Id', 'winPlacePerc']].to_csv('pytorch_adjusted.csv', index=False)